In [23]:
from langchain_groq import ChatGroq
from langchain.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms.ollama import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores.pinecone import Pinecone
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
import os

In [24]:
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [86]:
pdf_loader = PyPDFLoader('Dr Sanjay Sachdeva.pdf')
docs = pdf_loader.load()

In [87]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
documents = text_splitter.split_documents(docs)

In [88]:
embeddings = OllamaEmbeddings(
    model = "mxbai-embed-large"
)

In [104]:
from pinecone import Pinecone
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
db = PineconeVectorStore.from_documents(
    documents,
    embeddings,
    index_name = "langchain-index"
)

In [105]:
llm = ChatGroq(
    api_key=os.getenv('GROQ_API_KEY'),
    model='mixtral-8x7b-32768'
)

In [119]:
prompt = ChatPromptTemplate.from_template(
    '''
    Answer the question based on the provided context only:
    Context : {context}
    Question : {input}
    '''
)

In [120]:
document_chain = create_stuff_documents_chain(
    llm,
    prompt
)
retriever = db.as_retriever()

In [121]:
chain = create_retrieval_chain(
    retriever,
    document_chain
)

In [122]:
response = chain.invoke({
    'input':'''
               Provide all the test names from all the documents with their values in the given format below and also 
               provide the information if the test is positive or negative:
               TestName : Test Value (Positve or negative)
            ''',
})

In [123]:
print(response['answer'])

Based on the provided context, there is only one test result provided. Here is the result in the requested format:

TestName : Test Value (Positive or Negative)
ANTI NUCLEAR ANTIBODY / FACTOR (ANA/ANF), SERUM (EIA) : 13.03 Units (Negative)

Please note that the test value is 13.03 units, which is within the normal reference interval of less than 20.00 units. Therefore, the test result is interpreted as negative.
